# Setup
The following cells need to be run in order to use the Java kernel and set dependencies

In [ ]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

In [ ]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-simple</artifactId>
    <version>1.7.25</version>
  </dependency>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.4</version>
  </dependency>
  <dependency>
    <groupId>commons-io</groupId>
    <artifactId>commons-io</artifactId>
    <version>2.6</version>
  </dependency>
  <dependency>
    <groupId>com.googlecode.json-simple</groupId>
    <artifactId>json-simple</artifactId>
    <version>1.1.1</version>
</dependency>
</dependencies>

## Import dependencies, Connect to Aerospike

In [ ]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.BatchRecord;
import com.aerospike.client.BatchResults;
import com.aerospike.client.Bin;
import com.aerospike.client.Host;
import com.aerospike.client.Key;
import com.aerospike.client.Operation;
import com.aerospike.client.Record;
import com.aerospike.client.ResultCode;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapReturnType;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.exp.MapExp;
import com.aerospike.client.policy.Policy;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.query.Statement;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.task.ExecuteTask;

import java.util.List;
import java.util.Map;
import java.util.TreeMap;
import java.nio.charset.StandardCharsets;
import org.apache.commons.io.FileUtils;
import org.json.simple.JSONArray;
import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;

Host[] hosts = new Host[] {
    new Host("127.0.0.1", 3000)
};

WritePolicy writePolicy = new WritePolicy();
writePolicy.sendKey = true;

AerospikeClient client = new AerospikeClient(null, hosts);
System.out.println("Initialized Aerospike client and connected to the cluster.");

# JSON Document

## Document Format

The document is an array of objects like the example below:

```json
{
    points:"87",
    title:"Nicosia 2013 Vulkà Bianco (Etna)",
    description:"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
    taster: {
        name:"Kerin O’Keefe",
        twitter_handle:"@kerinokeefe"
    },
    price:null,
    designation:"Vulkà Bianco",
    variety:"White Blend",
    regions: {
        primary: "Etna",
        secondary: null
    },
    province:"Sicily & Sardinia",
    country:"Italy",
    winery:"Nicosia"
}
```

## Parse the JSON file

In [ ]:
String wineString = FileUtils.readFileToString(new File("./wine-data.json"), StandardCharsets.UTF_8);

Object obj = new JSONParser().parse(wineString);

JSONArray wines = (JSONArray) obj;

System.out.println("Document parsed");

# Write Data

The following code will create a record for each wine review in the data set.

In [ ]:
System.out.println("Writing records...\n");

for (int i = 0; i < wines.size(); i++) {
    Map map = (Map) wines.get(i);
    Map wineMap = new TreeMap<>();
    wineMap.putAll(map);

    Key key = new Key("demo", "wine-java", i);
    Bin wine = new Bin("wine_bin", wineMap, MapOrder.KEY_ORDERED);

    client.put(writePolicy, key, wine);
}

Key key = new Key("demo", "wine-java", 0);
Record record = client.get(null, key);
System.out.format("Wines: %2.100s\n\n", record.bins.get("wine_bin"));

System.out.println("Writing records complete");

# Read Data

### Read with a expression filter

In [ ]:
System.out.println("Reading with a filter expression...\n");
Expression exp = Exp.build(
    Exp.eq(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.STRING, Exp.val("country"), Exp.mapBin("wine_bin")),
            Exp.val("Georgia")
    )
);

QueryPolicy queryPolicy = new QueryPolicy();
queryPolicy.filterExp = exp;

Statement stmt = new Statement();
stmt.setNamespace("demo");
stmt.setSetName("wine-java");

RecordSet recordSet = client.query(queryPolicy, stmt);

while(recordSet.next()){
    Record record = recordSet.getRecord();
    System.out.format("Wine Info: %s\n\n", record.bins.get("wine_bin")); 
}

recordSet.close();
System.out.println("End of records returned");

### Create Secondary indexes

#### String index
Create a string index on the `country` map key.

In [ ]:
System.out.println("Creating country index...");
try {
    IndexTask task = client.createIndex(null,
                               "demo",
                               "wine-java",
                               "winery_idx",
                               "wine_bin",
                               IndexType.STRING,
                               IndexCollectionType.DEFAULT,
                               CTX.mapKey(Value.get("country")));
    task.waitTillComplete(1000, 0);
}
catch (AerospikeException ae) {
    if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
        throw ae;
    }
} 
System.out.println("Country index created");

#### Numeric index
Create an numeric index on the `price` map key.

In [ ]:
System.out.println("Creating price index...");
try {
    IndexTask task = client.createIndex(null,
                               "demo",
                               "wine-java",
                               "price_idx",
                               "wine_bin",
                               IndexType.NUMERIC,
                               IndexCollectionType.DEFAULT,
                               CTX.mapKey(Value.get("price")));
    task.waitTillComplete(1000, 0);
}
catch (AerospikeException ae) {
    if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
        throw ae;
    }
} 
System.out.println("Price index created");

#### Nested index
Create a string index on the `name` key in the `taster` map. 

In [ ]:
System.out.println("Creating taster index...");
try {
    IndexTask task = client.createIndex(null,
                               "demo",
                               "wine-java",
                               "taster_idx",
                               "wine_bin",
                               IndexType.STRING,
                               IndexCollectionType.DEFAULT,
                               CTX.mapKey(Value.get("taster")),
                               CTX.mapKey(Value.get("name")));
    task.waitTillComplete(1000, 0);
}
catch (AerospikeException ae) {
    if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
        throw ae;
    }
} 
System.out.println("Taster index created");

### Query data using a secondary index.
Return all the wines from the country Georgia.

In [ ]:
System.out.println("Reading with a string sindex filter...\n");
Statement stmt = new Statement();

stmt.setNamespace("demo");
stmt.setSetName("wine-java");

Filter filter = Filter.equal("wine_bin", "Georgia", CTX.mapKey(Value.get("country")));
stmt.setFilter(filter);

RecordSet recordSet = client.query(null, stmt);

while(recordSet.next()){
    Record record = recordSet.getRecord();
    System.out.format("Wine Info: %s\n\n", record.bins.get("wine_bin")); 
}

recordSet.close();
System.out.println("End of records returned");

Get all the documents for wines with a price between 500 and 1000 USD.

In [ ]:
System.out.println("Reading with a numeric sindex filter...\n");
Statement stmt = new Statement();

stmt.setNamespace("demo");
stmt.setSetName("wine-java");

Filter filter = Filter.range("wine_bin", 500, 1000, CTX.mapKey(Value.get("price")));
stmt.setFilter(filter);

RecordSet recordSet = client.query(null, stmt);

while(recordSet.next()){
    Record record = recordSet.getRecord();
    System.out.format("Wine Info: %s\n\n", record.bins.get("wine_bin")); 
}

recordSet.close();
System.out.println("End of records returned");

### Query data with a secondary index and filter expression

Get all documents for wines from France with more than 95 points and price less than $70

In [ ]:
System.out.println("Reading with a sindex and filter expression...\n");
QueryPolicy queryPolicy = new QueryPolicy();
queryPolicy.filterExp = Exp.build(
    Exp.and(
        Exp.gt(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("points"), Exp.mapBin("wine_bin")),
            Exp.val(95)
        ),
        Exp.lt(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("price"), Exp.mapBin("wine_bin")),
            Exp.val(70)
        )
    )
);

Statement stmt = new Statement();

stmt.setNamespace("demo");
stmt.setSetName("wine-java");

Filter filter = Filter.equal("wine_bin", "France", CTX.mapKey(Value.get("country")));
stmt.setFilter(filter);

RecordSet recordSet = client.query(queryPolicy, stmt);

while(recordSet.next()){
    Record record = recordSet.getRecord();
    System.out.format("Wine Info: %s\n\n", record.bins.get("wine_bin")); 
}

recordSet.close();
System.out.println("End of records returned");

# Update Data

## Create new wine object

In [ ]:
String newString = "{ " +
"    \"points\": 85," + 
"    \"title\": \"Barrel Racer 2013 Sauvignon Blanc (Solano County)\"," +
"    \"description\": \"Practically a liquid fruit salad, this smells and tastes like orange, apple, cantaloupe and banana. The texture is smooth and rounded, rather than crisp. A little buttery flavor comes through on the finish.\"," + 
"    \"taster\": {" +
"        \"name\":\"Jim Gordon\"," + 
"        \"twitter_handle\": \"@gordone_cellars\"" +
"        }," + 
"    \"price\": 18," + 
"    \"designation\": null," + 
"    \"variety\": \"Sauvignon Blanc\"," + 
"    \"regions\": {" +
"        \"primary\": \"Solano County\"," + 
"        \"secondary\": \"North Coast\"" +
"    }," + 
"    \"province\": \"California\"," + 
"    \"country\": \"US\"," + 
"    \"winery\": \"Barrel Racer\"}";
    
Object obj = new JSONParser().parse(newString);

Map newWine = (Map) obj;

System.out.println("New wine parsed");

### Update the collection with a new wine

In [ ]:
System.out.println("Adding new wine...\n");
Map wineMap = new TreeMap<>();
wineMap.putAll(newWine);

Key key = new Key("demo", "wine-java", 10001);
Bin wine = new Bin("wine_bin", wineMap, MapOrder.KEY_ORDERED);

client.put(writePolicy, key, wine);

Record record = client.get(null, key);

System.out.format("%s\n\n",record.bins);
System.out.println("New wine added\n");

### Update a wine within the collection

Update the price of a wine.

In [ ]:
System.out.println("Updating wine price...\n");
Key key = new Key("demo", "wine-java", 10001);

client.operate(writePolicy, key, MapOperation.put(MapPolicy.Default, "wine_bin", Value.get("price"), Value.get(24)));

Record record = client.get(null, key);

Map wineMap = (Map) record.bins.get("wine_bin");

System.out.println("winery: " + wineMap.get("winery") + ", title: " + wineMap.get("title") + ", price: " + wineMap.get("price"));

System.out.println("\nWine updated\n");

### Update all instances of a nested value in the collection

Update the twitter handle for Lauren Buzzeo on every wine they have tasted.

In [ ]:
System.out.println("Updating Twitter handle...\n");
Statement stmt = new Statement();

stmt.setNamespace("demo");
stmt.setSetName("wine-java");

Filter filter = Filter.equal("wine_bin", "Lauren Buzzeo", CTX.mapKey(Value.get("taster")), CTX.mapKey(Value.get("name")));
stmt.setFilter(filter);

ExecuteTask task = client.execute(null, stmt, 
    MapOperation.put(MapPolicy.Default, "wine_bin", Value.get("twitter_handle"), Value.get("laurenbuzzed"), CTX.mapKey(Value.get("taster")))
);

task.waitTillComplete();

Statement stmt = new Statement();
stmt.setNamespace("demo");
stmt.setSetName("wine-java");
stmt.setMaxRecords(5);

Filter filter = Filter.equal("wine_bin", "Lauren Buzzeo", CTX.mapKey(Value.get("taster")), CTX.mapKey(Value.get("name")));
stmt.setFilter(filter);

RecordSet recordSet = client.query(null, stmt);

while(recordSet.next()){
    Record record = recordSet.getRecord();
    Map bin = (Map)record.bins.get("wine_bin");
    System.out.format("Updated Info: %s\n", bin.get("taster")); 
}

recordSet.close();

System.out.println("\nTwitter handle updated");

# Delete Data

Delete all wines with a points vaule less than 81.

In [ ]:
System.out.println("Deleting records...");
Expression exp = Exp.build(
    Exp.lt(
        MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("points"), Exp.mapBin("wine_bin")),
        Exp.val(81)
    )
);

WritePolicy writePolicy = new WritePolicy();
writePolicy.filterExp = exp;

Statement stmt = new Statement();
stmt.setNamespace("demo");
stmt.setSetName("wine-java");

ExecuteTask task = client.execute(writePolicy, stmt, Operation.delete());

task.waitTillComplete(1000, 0);

System.out.println("Checking for records...");

QueryPolicy queryPolicy = new QueryPolicy();
queryPolicy.filterExp = exp;

Statement stmt = new Statement();
stmt.setNamespace("demo");
stmt.setSetName("wine-java");

RecordSet recordSet = client.query(queryPolicy, stmt);

if(!recordSet.next()){
    System.out.println("Records deleted");
}

## Cleanup

In [ ]:
// Remove the data
client.truncate(null, "demo", "wine-java", null);
System.out.println("Data removed");

// Remove the secondary indexes
IndexTask taskCountry = client.dropIndex(null,
    "demo",
    "wine-java", 
    "country_idx"
);
taskCountry.waitTillComplete();
System.out.println("Country index removed");

IndexTask taskPrice = client.dropIndex(null,
    "demo",                     
    "wine-java",
    "price_idx"
);
taskPrice.waitTillComplete();
System.out.println("Price index removed");

IndexTask taskTaster = client.dropIndex(null,
    "demo",
    "wine-java",
    "taster_idx"
);
taskTaster.waitTillComplete();
System.out.println("Taster index removed");

client.close();